In [1]:
import pandas as pd 

In [21]:
df = pd.read_csv('spam.csv',engine="python")
df.dropna(inplace = True,axis=1)
df.head() 

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [22]:
df['v2'][99]

"Please don't text me anymore. I have nothing else to say."

In [23]:
df.shape

(5572, 2)

In [26]:
# 1= spam 0= ham
for i in range(len(df['v1'])):
    if df['v1'][i] == 'ham':
        df['v1'][i] = 0
    else:
        df['v1'][i] = 1

In [27]:
df.head() 

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
X = df['v2']
y = df['v1']

In [32]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [33]:
ps = PorterStemmer()

In [40]:
## Data Preprocessing
corpus = []
for i in range(len(X)):
    review = re.sub('[^a-zA-Z]'," ",X[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = " ".join(review)
    corpus.append(review)

### One Hot Representation

In [44]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [48]:
voc_size = 5000
onehot_rep = [one_hot(words,voc_size) for words in corpus]

In [95]:
## Embedding word metric

sent_size = 50
embedded_doc = pad_sequences(onehot_rep,sent_size)

embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 2872, 3935, 4657, 4936, 4407,  154, 3461, 1035, 4863, 4609,
       3683, 2189,  178,  150, 1159, 4116])

In [51]:
## Model creation
dim = 50
model = Sequential()
model.add(Embedding(voc_size,dim,input_length = sent_size))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(  optimizer='adam',
    loss="binary_crossentropy",
    metrics=['accuracy'],)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            250000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________


In [106]:
import numpy as np
X_final = np.array(embedded_doc).astype('int32')
y_final = np.array(y).astype('int32')

print(X_final.shape,y_final.shape)

(5572, 50) (5572,)


In [107]:
from sklearn.model_selection import train_test_split

In [108]:
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.2,random_state=50)

In [109]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=64,epochs=10)

Epoch 1/10
70/70 [==============================] - 20s 135ms/step - loss: 0.2861 - accuracy: 0.9071 - val_loss: 0.1172 - val_accuracy: 0.9695
Epoch 2/10
70/70 [==============================] - 8s 107ms/step - loss: 0.0537 - accuracy: 0.9847 - val_loss: 0.0686 - val_accuracy: 0.9830
Epoch 3/10
70/70 [==============================] - 5s 78ms/step - loss: 0.0284 - accuracy: 0.9921 - val_loss: 0.0720 - val_accuracy: 0.9812
Epoch 4/10
70/70 [==============================] - 6s 84ms/step - loss: 0.0180 - accuracy: 0.9957 - val_loss: 0.0677 - val_accuracy: 0.9848
Epoch 5/10
70/70 [==============================] - 6s 81ms/step - loss: 0.0091 - accuracy: 0.9978 - val_loss: 0.0802 - val_accuracy: 0.9839
Epoch 6/10
70/70 [==============================] - 6s 80ms/step - loss: 0.0057 - accuracy: 0.9984 - val_loss: 0.1057 - val_accuracy: 0.9785
Epoch 7/10
70/70 [==============================] - 6s 85ms/step - loss: 0.0044 - accuracy: 0.9989 - val_loss: 0.0875 - val_accuracy: 0.9848
Epoch 8/10

In [110]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [113]:
y_pred = model.predict_classes(X_test)
confusion_matrix(y_test,y_pred)

C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[954,   0],
       [ 18, 143]], dtype=int64)

In [114]:
accuracy_score(y_test,y_pred)

0.9838565022421525